## Source code

https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/openvqa/core/base_cfgs.py
https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/utils/exec.py#L36
https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/utils/test_engine.py#L49
https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/openvqa/datasets/vqa/vqa_loader.py#L68

# Load dataset

In [2]:
import gc
import json

import numpy as np
import torch
import torch.utils.data as Data
import yaml
# from yaml import CLoader

from openvqa.datasets.dataset_loader import DatasetLoader, EvalLoader
from openvqa.datasets.vqa.eval.vqa import VQA
from openvqa.datasets.vqa.eval.vqaEval import VQAEval
from openvqa.models.model_loader import CfgLoader, ModelLoader
from run import create_parser
from utils.test_engine import test_engine

parser = create_parser()
arg = '--RUN test --MODEL mcan_large --DATASET vqa --NW 2 --CKPT_V mcan_large --CKPT_E 13'
args = parser.parse_args(arg.split(' '))

cfg_file = "configs/{}/{}.yml".format(args.DATASET, args.MODEL)
with open(cfg_file, 'r') as f:
    # yaml_dict = yaml.load(f, Loader=CLoader)
    yaml_dict = yaml.load(f)

__C = CfgLoader(yaml_dict['MODEL_USE']).load()
args = __C.str_to_bool(args)
args_dict = __C.parse_to_dict(args)

args_dict = {**yaml_dict, **args_dict}
__C.add_args(args_dict)
__C.proc()

print('Hyper Parameters:')
print(__C)

Checking dataset ........
Finished!

Hyper Parameters:
{ BATCH_SIZE        }->64
{ BBOXFEAT_EMB_SIZE }->2048
{ BBOX_NORMALIZE    }->True
{ CACHE_PATH        }->./results/cache
{ CKPTS_PATH        }->./ckpts
{ CKPT_EPOCH        }->13
{ CKPT_PATH         }->None
{ CKPT_VERSION      }->mcan_large
{ DATASET           }->vqa
{ DATA_PATH         }->{'vqa': './data/vqa', 'gqa': './data/gqa', 'clevr': './data/clevr'}
{ DATA_ROOT         }->./data
{ DEVICES           }->[0]
{ DROPOUT_R         }->0.1
{ EVAL_BATCH_SIZE   }->16
{ EVAL_EVERY_EPOCH  }->False
{ FEATS_PATH        }->{'vqa': {'train': './data/vqa/feats/train2014', 'val': './data/vqa/feats/val2014', 'test': './data/vqa/feats/test2015'}, 'gqa': {'default-frcn': './data/gqa/feats/gqa-frcn', 'default-grid': './data/gqa/feats/gqa-grid'}, 'clevr': {'train': './data/clevr/feats/train', 'val': './data/clevr/feats/val', 'test': './data/clevr/feats/test'}}
{ FEAT_SIZE         }->{'vqa': {'FRCN_FEAT_SIZE': (100, 2048), 'BBOX_FEAT_SIZE': (100, 5)

<ipython-input-2-4222a2251f52>:26: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_dict = yaml.load(f)


In [3]:
dataset = DatasetLoader(__C).DataSet()

data_size = dataset.data_size
token_size = dataset.token_size
ans_size = dataset.ans_size
pretrained_emb = dataset.pretrained_emb

/home/user/.conda/envs/openvqa/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_vectors_web_lg' (2.1.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


 ========== Dataset size: 447793


/home/user/.conda/envs/openvqa/lib/python3.8/site-packages/spacy/_ml.py:287: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (1070971, 300))
  warnings.warn(Warnings.W020.format(shape=vectors.data.shape))


 ========== Question token vocab size: 20573
 ========== Answer token vocab size (occur more than 8 times): 3129
Finished!



In [4]:
net = ModelLoader(__C).Net(
    __C,
    pretrained_emb,
    token_size,
    ans_size
)

net.cuda()
net.eval()

with open('ckpts/ckpt_mcan_large/epoch13.pkl', 'rb') as f:
    state = torch.load(f)
net.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [5]:
dataloader = Data.DataLoader(
    dataset,
    batch_size=__C.EVAL_BATCH_SIZE,
    shuffle=False,
    num_workers=__C.NUM_WORKERS,
    pin_memory=__C.PIN_MEM
)

# Prediction (on test set)

In [6]:
%%time
ans_ix_list = []
pred_list = []

for step, (
        frcn_feat_iter,
        grid_feat_iter,
        bbox_feat_iter,
        ques_ix_iter,
        ans_iter
    ) in enumerate(dataloader):
    print("\rEvaluation: [step %4d/%4d]" % (
        step,
        int(data_size / __C.EVAL_BATCH_SIZE),
    ), end='          ')

    frcn_feat_iter = frcn_feat_iter.cuda()
    grid_feat_iter = grid_feat_iter.cuda()
    bbox_feat_iter = bbox_feat_iter.cuda()
    ques_ix_iter = ques_ix_iter.cuda()

    pred = net(
        frcn_feat_iter,
        grid_feat_iter,
        bbox_feat_iter,
        ques_ix_iter
    )

    pred_np = pred.cpu().data.numpy()
    pred_argmax = np.argmax(pred_np, axis=1)
    
    if pred_argmax.shape[0] != __C.EVAL_BATCH_SIZE:
        pred_argmax = np.pad(
            pred_argmax,
            (0, __C.EVAL_BATCH_SIZE - pred_argmax.shape[0]),
            mode='constant',
            constant_values=-1
        )

    ans_ix_list.append(pred_argmax)
    
    if __C.TEST_SAVE_PRED:
        if pred_np.shape[0] != __C.EVAL_BATCH_SIZE:
            pred_np = np.pad(
                pred_np,
                ((0, __C.EVAL_BATCH_SIZE - pred_np.shape[0]), (0, 0)),
                mode='constant',
                constant_values=-1
            )

        pred_list.append(pred_np)

# Evaluation: [step 6698/6698]
# CPU times: user 14h 57min 55s, sys: 19min 55s, total: 15h 17min 51s
# Wall time: 7h 40min 34s
# Evaluation: [step 13993/13993]
# CPU times: user 13min 48s, sys: 8min 59s, total: 22min 47s
# Wall time: 35min 22s

Evaluation: [step 27987/27987]          CPU times: user 24min 12s, sys: 6min 10s, total: 30min 22s
Wall time: 27min 12s


In [7]:
ans_ix_list = np.array(ans_ix_list).reshape(-1)

qid_list = [ques['question_id'] for ques in dataset.ques_list]
ans_size = dataset.ans_size

result = [{
        'answer': dataset.ix_to_ans[str(ans_ix_list[qix])],
        'question_id': int(qid_list[qix])
    } for qix in range(qid_list.__len__())]

result_eval_file = __C.CACHE_PATH + '/result_run_' + __C.VERSION
log_file = __C.LOG_PATH + '/log_run_' + __C.VERSION + '.txt'
result_eval_file += '.json'
print('Save the result to file: {}'.format(result_eval_file))
with open(result_eval_file, 'w') as f:
    json.dump(result, f)

Save the result to file: ./results/cache/result_run_1950821.json


# Evaluation (on validation set)

In [ ]:
result_eval_file = __C.CACHE_PATH + '/result_run_' + __C.VERSION
result_eval_file += '.json'

ques_file_path = __C.RAW_PATH[__C.DATASET][__C.SPLIT['val']]
ans_file_path = __C.RAW_PATH[__C.DATASET][__C.SPLIT['val'] + '-anno']
# uncomment line 165-166 in http://localhost:8888/edit/openvqa/openvqa/datasets/vqa/eval/vqa.py
vqa = VQA(ans_file_path, ques_file_path)
vqaRes = vqa.loadRes(result_eval_file, ques_file_path)

loading VQA annotations and questions into memory...
0:00:05.052310
creating index...
index created!
Loading and preparing results...     
DONE (t=0.46s)
creating index...
index created!


In [ ]:
vqaEval = VQAEval(vqa, vqaRes, n=2)

# quesIds = [d['question_id'] for d in result]
# vqaEval.evaluate(quesIds)
vqaEval.evaluate()

computing accuracy
Finished Percent: [####################] 99% Done computing accuracy


In [ ]:
vqaEval.accuracy

{'overall': 84.09,
 'perQuestionType': {'none of the above': 82.19,
  'what are the': 75.57,
  'what is': 75.66,
  'what': 69.0,
  'is this a': 97.88,
  'is this': 97.46,
  'what is the man': 84.12,
  'how many': 75.92,
  'what does the': 48.29,
  'why': 46.35,
  'is it': 98.84,
  'why is the': 47.41,
  'what color is the': 92.15,
  'is there a': 98.67,
  'is the': 96.79,
  'is that a': 97.79,
  'are these': 96.65,
  'are the': 96.26,
  'what is the': 75.58,
  'which': 68.28,
  'could': 98.61,
  'are there': 98.21,
  'what kind of': 76.9,
  'has': 97.16,
  'what color are the': 91.07,
  'are there any': 99.08,
  'is this person': 98.05,
  'does the': 97.25,
  'where is the': 60.6,
  'how many people are': 74.58,
  'can you': 97.58,
  'what type of': 77.29,
  'what is the color of the': 92.4,
  'what is on the': 76.11,
  'does this': 97.48,
  'is this an': 97.49,
  'who is': 63.25,
  'what is this': 84.76,
  'is there': 98.29,
  'are': 95.61,
  'what time': 45.95,
  'how': 54.52,
  'wha